In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("C:/Class/MLproject/train3.csv")

In [3]:
from sklearn.model_selection import train_test_split

X = train[['id','bus_route_id','latitude','longitude', '6~7_ride','7~8_ride',
                 '8~9_ride','9~10_ride','10~11_ride','11~12_ride','6~7_takeoff',
                 '7~8_takeoff','8~9_takeoff','9~10_takeoff','10~11_takeoff','11~12_takeoff','region_e',
       'region_n', 'region_o', 'region_s', 'region_w', 'in_out_시내',
       'in_out_시외', 'rain_0', 'rain_1', 'rain_2', 'rain_3']]


y = train[['18~20_ride']]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)

### ElasticNet 모델

ElasticNet 모델이란 Ridge와 Lasso를 결합한 형태의 모델  
즉 가중치의 절대값의 합(L1)과 제곱의 합(L2)를 동시에 제약조건으로 가지는 모델  
L1과 L2의 비율을 파라미터 l1_ratio를 통해 지정할 수 있음

In [13]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

# Use grid search to tune the parameters:

parametersGrid = {"max_iter": [1, 5, 10],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1)}

eNet = ElasticNet()
grid = GridSearchCV(eNet, parametersGrid, cv=10)
grid.fit(X_train, y_train)
y_predict = grid.predict(X_test)

C:\Users\KWAK\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.371e+04, tolerance: 3.993e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\KWAK\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.373e+04, tolerance: 4.002e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_

In [15]:
print(grid.best_score_) #최상의 교차 검증 정확도
print(grid.score(X_test, y_test)) #테스트 세트 점수
print(grid.best_params_) #최적의 매개변수

0.41481912477088045
0.412029247012875
{'alpha': 0.001, 'l1_ratio': 0.0, 'max_iter': 10}


In [16]:
elasticnet = ElasticNet(alpha = 0.001, l1_ratio = 0.0, max_iter = 10).fit(X_train, y_train)

C:\Users\KWAK\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.303e+04, tolerance: 4.442e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


최적의 L1비율이 0.0이면 결국 L2를 사용하는 ridge랑 같은 건가..?

In [17]:
print(elasticnet.score(X_train, y_train))
print(elasticnet.score(X_test, y_test))

0.4149969341844999
0.412029247012875


In [18]:
from dmba import regressionSummary
regressionSummary(y_test, elasticnet.predict(X_test))


Regression statistics

               Mean Error (ME) : 0.0009
Root Mean Squared Error (RMSE) : 0.3001
     Mean Absolute Error (MAE) : 0.2169


### XGBoost

In [21]:
!pip install xgboost

In [29]:
import xgboost as xgb

In [36]:
from xgboost import plot_importance
from xgboost import XGBRegressor
import xgboost
from sklearn.metrics import explained_variance_score
import matplotlib.pyplot as plt


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)
print(X_train.shape, X_test.shape)

(290796, 27) (124627, 27)


#### XGBoost 파라미터
* max_depth [default = 6] : 나무의 최대 깊이
* min_child_weight [default = 1] : child node에 필요한 인스턴스 가중치의 최소 합계
* gamma [default = 0] : 트리의 리프 노드에서 추가 파티션을 만드는 데 필요한 최소 손실 감소
* subsample [default = 1] : 훈련 인스턴스의 하위 샘플 비율
* colsample_bytree [default = 1] : 각 트리를 구성할 때 열의 하위 샘플 비율
* eta [default = 0.3] : 과적합을 방지하기 위해 업데이트에 사용되는 단계 크기 축소
* num_round : 부스팅을 위한 반복 학습 수

더 있는 듯 ..?

In [31]:
xgb_model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.08, gamma=0, subsample=0.75,
                                colsample_bytree=1, max_depth=7)

xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [32]:
print(xgb_model.score(X_train, y_train))
print(xgb_model.score(X_test, y_test))

0.693875144177528
0.5983576960451836


In [33]:
from dmba import regressionSummary
regressionSummary(y_test, xgb_model.predict(X_test))


Regression statistics

               Mean Error (ME) : -0.0007
Root Mean Squared Error (RMSE) : 0.2483
     Mean Absolute Error (MAE) : 0.1697


In [ ]:
from sklearn.model_selection import GridSearchCV

# 객체 생성, 일단은 트리 100개만 만듦
xgb_model = XGBRegressor(n_estimators=1000)

# 후보 파라미터 선정
params = {'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set=[(X_test, y_test)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)


[0]	validation_0-auc:0.81461


C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-auc:0.82234
[2]	validation_0-auc:0.82693
[3]	validation_0-auc:0.83664
[4]	validation_0-auc:0.84690
[5]	validation_0-auc:0.84873
[6]	validation_0-auc:0.85190
[7]	validation_0-auc:0.85257
[8]	validation_0-auc:0.85487
[9]	validation_0-auc:0.85543
[10]	validation_0-auc:0.85980
[11]	validation_0-auc:0.86093
[12]	validation_0-auc:0.86151
[13]	validation_0-auc:0.86487
[14]	validation_0-auc:0.86510
[15]	validation_0-auc:0.86590
[16]	validation_0-auc:0.86698
[17]	validation_0-auc:0.86917
[18]	validation_0-auc:0.87122
[19]	validation_0-auc:0.87220
[20]	validation_0-auc:0.87311
[21]	validation_0-auc:0.87375
[22]	validation_0-auc:0.87556
[23]	validation_0-auc:0.87610
[24]	validation_0-auc:0.87705
[25]	validation_0-auc:0.87763
[26]	validation_0-auc:0.87844
[27]	validation_0-auc:0.87941
[28]	validation_0-auc:0.87979
[29]	validation_0-auc:0.87996
[30]	validation_0-auc:0.88054
[31]	validation_0-auc:0.88077
[32]	validation_0-auc:0.88118
[33]	validation_0-auc:0.88162
[34]	validation_0-a

C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.81469
[1]	validation_0-auc:0.82277
[2]	validation_0-auc:0.82732
[3]	validation_0-auc:0.83726
[4]	validation_0-auc:0.84754
[5]	validation_0-auc:0.84956
[6]	validation_0-auc:0.85239
[7]	validation_0-auc:0.85315
[8]	validation_0-auc:0.85536
[9]	validation_0-auc:0.85579
[10]	validation_0-auc:0.85987
[11]	validation_0-auc:0.86143
[12]	validation_0-auc:0.86212
[13]	validation_0-auc:0.86514
[14]	validation_0-auc:0.86536
[15]	validation_0-auc:0.86655
[16]	validation_0-auc:0.86742
[17]	validation_0-auc:0.86986
[18]	validation_0-auc:0.87143
[19]	validation_0-auc:0.87210
[20]	validation_0-auc:0.87309
[21]	validation_0-auc:0.87489
[22]	validation_0-auc:0.87541
[23]	validation_0-auc:0.87547
[24]	validation_0-auc:0.87621
[25]	validation_0-auc:0.87712
[26]	validation_0-auc:0.87771
[27]	validation_0-auc:0.87810
[28]	validation_0-auc:0.87861
[29]	validation_0-auc:0.87917
[30]	validation_0-auc:0.87966
[31]	validation_0-auc:0.88141
[32]	validation_0-auc:0.88213
[33]	validation_0-au

...

[1]	validation_0-auc:0.82234
[2]	validation_0-auc:0.82693
[3]	validation_0-auc:0.83664
[4]	validation_0-auc:0.84690
[5]	validation_0-auc:0.84873
[6]	validation_0-auc:0.85190
[7]	validation_0-auc:0.85257
[8]	validation_0-auc:0.85487
[9]	validation_0-auc:0.85543
[10]	validation_0-auc:0.85979
[11]	validation_0-auc:0.86093
[12]	validation_0-auc:0.86151
[13]	validation_0-auc:0.86488
[14]	validation_0-auc:0.86510
[15]	validation_0-auc:0.86588
[16]	validation_0-auc:0.86697
[17]	validation_0-auc:0.86917
[18]	validation_0-auc:0.87123
[19]	validation_0-auc:0.87220
[20]	validation_0-auc:0.87306
[21]	validation_0-auc:0.87435
[22]	validation_0-auc:0.87489
[23]	validation_0-auc:0.87523
[24]	validation_0-auc:0.87620
[25]	validation_0-auc:0.87670
[26]	validation_0-auc:0.87760
[27]	validation_0-auc:0.87821
[28]	validation_0-auc:0.87866
[29]	validation_0-auc:0.87912
[30]	validation_0-auc:0.88118
[31]	validation_0-auc:0.88196
[32]	validation_0-auc:0.88265
[33]	validation_0-auc:0.88280
[34]	validation_0-a

C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-auc:0.82277
[2]	validation_0-auc:0.82732
[3]	validation_0-auc:0.83726
[4]	validation_0-auc:0.84755
[5]	validation_0-auc:0.84956
[6]	validation_0-auc:0.85240
[7]	validation_0-auc:0.85316
[8]	validation_0-auc:0.85536
[9]	validation_0-auc:0.85580
[10]	validation_0-auc:0.85987
[11]	validation_0-auc:0.86144
[12]	validation_0-auc:0.86213
[13]	validation_0-auc:0.86516
[14]	validation_0-auc:0.86537
[15]	validation_0-auc:0.86658
[16]	validation_0-auc:0.86753
[17]	validation_0-auc:0.86981
[18]	validation_0-auc:0.87250
[19]	validation_0-auc:0.87302
[20]	validation_0-auc:0.87402
[21]	validation_0-auc:0.87518
[22]	validation_0-auc:0.87602
[23]	validation_0-auc:0.87686
[24]	validation_0-auc:0.87782
[25]	validation_0-auc:0.87873
[26]	validation_0-auc:0.87941
[27]	validation_0-auc:0.87995
[28]	validation_0-auc:0.88022
[29]	validation_0-auc:0.88042
[30]	validation_0-auc:0.88081
[31]	validation_0-auc:0.88109
[32]	validation_0-auc:0.88121
[33]	validation_0-auc:0.88143
[34]	validation_0-a

C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\KWAK\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.81480
[1]	validation_0-auc:0.82285
[2]	validation_0-auc:0.82678
[3]	validation_0-auc:0.83724
[4]	validation_0-auc:0.84401
[5]	validation_0-auc:0.84601
[6]	validation_0-auc:0.84955
[7]	validation_0-auc:0.85031
[8]	validation_0-auc:0.85246
[9]	validation_0-auc:0.85300
[10]	validation_0-auc:0.85876
[11]	validation_0-auc:0.86036
[12]	validation_0-auc:0.86095
[13]	validation_0-auc:0.86432
[14]	validation_0-auc:0.86454
[15]	validation_0-auc:0.86584
[16]	validation_0-auc:0.86719
[17]	validation_0-auc:0.86944
[18]	validation_0-auc:0.87103
[19]	validation_0-auc:0.87190
[20]	validation_0-auc:0.87261
[21]	validation_0-auc:0.87416
[22]	validation_0-auc:0.87461
[23]	validation_0-auc:0.87547
[24]	validation_0-auc:0.87627
[25]	validation_0-auc:0.87690
[26]	validation_0-auc:0.87852
[27]	validation_0-auc:0.87938
[28]	validation_0-auc:0.87986
[29]	validation_0-auc:0.88009
[30]	validation_0-auc:0.88062
[31]	validation_0-auc:0.88080
[32]	validation_0-auc:0.88172
[33]	validation_0-au

최적의 파라미터 다시 찾아볼 것 .. 